# PAULI-DECOMPOSITION for PolSAR 

Copyright 2025 European Space Agency (ESA)

Licensed under  ESA Software Community License Permissive (Type 3) – v2.4

## Setup 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

### Prerequisities

In [ ]:
import pathlib

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import bioqlk

### Image processing parameters

In [ ]:
winsz = [20, 4]  # Window size for smoothing 
gamma = 1 # Power scaling factor: Default (no scaling is for gamma = 1) If gamma > 1, brighter values are enhanced, if gamma < 1, darker values are enhanced 

threshold = 0.03
# threshold = 0

# Phase correction of 90 degrees as standard, can adjusted per channel: 
# pc_hh = np.deg2rad(-23.75)
# pc_hv = np.deg2rad(113.46)
# pc_vh = np.deg2rad(-89.7)

# pc_hh = np.deg2rad(+23.75)
# pc_hv = np.deg2rad(-113.46)
# pc_vh = np.deg2rad(+89.7)

pc_hh = np.deg2rad(+90)  # np.pi/2
pc_hv = np.deg2rad(+90)  # np.pi/2
pc_vh = np.deg2rad(-90)  # - np.pi/2

# pc_hh = 0
# pc_hv = 0
# pc_vh = 0

### Read image
Import image and read channels.

In [ ]:
root = pathlib.Path("~/data/biomass/first-image/SLCs/MED").expanduser()
basepaths = [
    root / "BIO_S1_SCS__1S_20250523T102700_20250523T102852_C_G___M___C___T____F001_01_D9DXYJ",
    root / "BIO_S1_SCS__1S_20250523T102835_20250523T103027_C_G___M___C___T____F001_01_D9E0GM",
]

## Step 1: Compute the scaling values

In [ ]:
def plot_hist(red, green, blue, title="Pauli", scale="dB", fig=None, ax=None):
    if fig is None and ax is None:
        fig, ax = plt.subplots(3, sharex=True)

    ax[0].hist(red.ravel(), 500, label="k_1")
    # ax[0].set_title("k_1")
    ax[0].grid()
    ax[0].legend()
    ax[0].set_title(title)

    ax[1].hist(green.ravel(), 500, label="k_2")
    # ax[1].set_title("k_2")
    ax[1].grid()
    ax[1].legend()

    ax[2].hist(blue.ravel(), 500, label="k_3")
    # ax[2].set_title("k_3")
    ax[2].grid()
    ax[2].legend()
    ax[2].set_xlabel(scale)

    return fig, ax

In [ ]:
import collections

scaling_data = collections.defaultdict(list)
fig, ax = plt.subplots(3, len(basepaths), sharex=True)
for idx, basepath in enumerate(basepaths):
    name = basepath.name
    hh, hv, vh, vv, meta, gcps = bioqlk.load_data(basepath, return_metadata=True)
    
    # Apply the phase correction
    if [pc_hh, pc_hv, pc_vh, 0.] != [0., 0., 0., 0.]:
        hh, hv, vh, vv = bioqlk.fix_ical_phase(
            hh, hv, vh, vv, pc_hh, pc_hv, pc_vh, 0.
        )

    red, green, blue = bioqlk.pauli(hh, hv, vh, vv, scale=bioqlk.EScale.DB)

    red = bioqlk.smooth_nan(red, winsz)
    green = bioqlk.smooth_nan(green, winsz)
    blue = bioqlk.smooth_nan(blue, winsz)
    
    red_low, red_high = bioqlk.quantile_scaling(red, threshold=threshold)
    green_low, green_high = bioqlk.quantile_scaling(green, threshold=threshold)
    blue_low, blue_high = bioqlk.quantile_scaling(blue, threshold=threshold)

    print(name)
    print(f"RED: ({red_low}, {red_high})")
    print(f"GREEN: ({green_low}, {green_high})")
    print(f"BLUE: ({blue_low}, {blue_high})")

    scaling_data["red_low"].append(red_low)
    scaling_data["red_high"].append(red_high)
    scaling_data["green_low"].append(green_low)
    scaling_data["green_high"].append(green_high)
    scaling_data["blue_low"].append(blue_low)
    scaling_data["blue_high"].append(blue_high)

    if True:
        plot_hist(red, green, blue, fig=fig, ax=ax[:, idx])

In [ ]:
scaling_data

In [ ]:
scaling_params = {}

scaling_params["red_low"] = float(np.min(scaling_data["red_low"]))
scaling_params["red_high"] = float(np.max(scaling_data["red_high"]))
scaling_params["green_low"] = float(np.min(scaling_data["green_low"]))
scaling_params["green_high"] = float(np.max(scaling_data["green_high"]))
scaling_params["blue_low"] = float(np.min(scaling_data["blue_low"]))
scaling_params["blue_high"] = float(np.max(scaling_data["blue_high"]))

scaling_params

## Step 3: Image Processing

In [ ]:
representation = "pauli"
outdir = pathlib.Path("tmp-align")
outdir.mkdir(exist_ok=True, parents=True)

for basepath in basepaths:
    print(basepath.name)

    bioqlk.make_rgb(
        basepath,
        # outfile,
        representation=representation,
        # scale: EScale = EScale.DB,
        # alpha: int | bool | None = None,
        scaling_method=bioqlk.EScalingMethod.MANUAL,
        # kscale: float | None = None,
        scaling_ranges=scaling_params,
        # winsize: tuple[int, int] = (20, 4),
        # decimate: bool = False,
        # quick_test: bool = False,
        phase_correction=[pc_hh, pc_hv, pc_vh, 0],
        # float_file: str | None = None,
        kmz_file=True,
        # kmz_res: float = 0.001,
        # png_file: str | None = None,
        outdir=outdir,
    )
print("done")

(p313) antonio.valentino@ESR01114333 tmp % python3 ../bioqlk.py ~/data/biomass/first-image/SLCs/MED/BIO_S1_SCS__1S_20250523T102700_20250523T102852_C_G___M___C___T____F001_01_D9DXYJ --phase-corrections 90 90 -90 0 --kmz --outdir . 
INFO: scale_to_8bits - thresholds: [0.3, 0.7]
INFO: scale_to_8bits - [low, high]: [-2.898, -1.615]
INFO: scale_to_8bits - thresholds: [0.3, 0.7]
INFO: scale_to_8bits - [low, high]: [-19.333, -17.953]
INFO: scale_to_8bits - thresholds: [0.3, 0.7]
INFO: scale_to_8bits - [low, high]: [-4.330, -3.063]
Using band 4 of source image as alpha.
Creating output file that is 11338P x 39083L.
0...10...20...30...40...50...60...70...80...90...100 - done.